In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 创建 Vertex AI Vector Search 索引

在左侧表格中:
<center>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
</center>

<center>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fvector_search%2Fsdk_vector_search_for_indexing.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
</center>

<center>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
</center>

<center>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
</center>

## 概述

此示例演示如何使用Vertex AI ANN服务。这是一个大规模、低延迟的解决方案，用于查找大语料库中相似向量（或更具体地说是“嵌入”）。此外，这是一个完全托管的服务，进一步减少了运营开销。Vertex AI ANN服务是基于谷歌研究开发的[近似最近邻（ANN）技术](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html)构建的。

了解更多关于[Vertex AI向量搜索](https://cloud.google.com/vertex-ai/docs/vector-search/overview)。

### 目标

在这个笔记本中，您将学习如何创建近似最近邻居（ANN）索引，对索引进行查询，并验证索引的性能。

本教程使用以下 Vertex AI 服务：

- Vertex AI 矢量搜索

执行的步骤包括：

* 创建ANN索引和蛮力索引。
* 使用VPC网络创建索引端点。
* 部署ANN索引和蛮力索引。
* 执行在线查询。
* 计算召回率。

数据集

本教程使用的数据集是[GloVe数据集](https://nlp.stanford.edu/projects/glove/)。

GloVe是一种用于获取单词向量表示的无监督学习算法。训练是在语料库中聚合的全局单词共现统计数据上进行的。所得到的表示展现了单词向量空间的有趣的线性子结构。

开始吧

### 为Python安装Vertex AI SDK和其他所需的软件包

In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage \
                         grpcio-tools \
                         h5py

### 重新启动运行时（仅适用于Colab）

要使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。在继续下一步之前，请等待它完成。⚠️</b>
</div>

### 认证您的笔记本环境（仅限Colab）

在Google Colab上认证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息
了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

准备一个VPC网络
为了减少可能导致不必要的增加延迟的网络开销，最好通过直接的VPC对等连接从您的VPC调用ANN终端节点。 
* 如果您还没有VPC Peering连接，下面的部分将描述如何设置它。
* 这是一个一次性的初始设置任务。您也可以重用现有的VPC网络并跳过这个部分。

In [ ]:
VPC_NETWORK = "[your-vpc-network-name]"  # @param {type:"string"}

PEERING_RANGE_NAME = "ann-haystack-range"

In [ ]:
import os

# Remove the if condition to run the encapsulated code
if not os.getenv("IS_TESTING"):
    # Create a VPC network
    ! gcloud compute networks create {VPC_NETWORK} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

    # Add necessary firewall rules
    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-icmp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow icmp

    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-internal --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-rdp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-ssh --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:22

    # Reserve IP range
    ! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={VPC_NETWORK} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range"

    # Set up peering with service networking
    # Your account must have the "Compute Network Admin" role to run the following.
    ! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={VPC_NETWORK} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

身份验证：当您注销并且需要重新获取凭据时，在 Vertex AI Workbench 笔记本终端重新运行 `gcloud auth login` 命令。

警告：`MatchingIndexEndpoint.match`方法（用于针对部署的索引创建在线查询）必须在满足以下要求的 Vertex AI 工作台笔记本实例中执行：
* **与您的 ANN 服务部署在相同地区**（例如，如果您将 `LOCATION = "us-central1"` 设置为和教程相同，则笔记本实例必须在 `us-central1` 中）。

* **确保选择您为 ANN 服务创建的 VPC 网络**（而不是使用“默认”）。也就是说，您必须创建一个使用您之前创建的 VPC 网络的新笔记本实例。从那个笔记本实例运行教程的其余部分。
* 如果您在不同的 VPC 网络或地区的 Colab 或 Vertex AI 工作台笔记本中运行，预期会在“创建在线查询”部分失败。

导入所需的库

In [ ]:
import json

import h5py
from google.cloud import aiplatform
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import \
    Namespace

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

初始化 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

准备数据

GloVe数据集包含一组预训练的嵌入。这些嵌入被分成“训练”部分和“测试”部分。您可以从“训练”部分创建一个向量搜索索引，并使用“测试”部分中的嵌入向量作为查询向量来测试向量搜索索引。

**注意:** 虽然数据分割使用术语“训练”，但这些是预训练的嵌入，因此可以直接用于搜索索引。术语“训练”和“测试”分割仅用于与机器学习术语保持一致。

下载GloVe数据集。

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5 .

将数据读入内存。

In [ ]:
# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 10

h5 = h5py.File("glove-100-angular.hdf5", "r")
train = h5["train"]
test = h5["test"]

In [ ]:
# check the first record
train[0]

将火车拆分保存为JSONL格式。

数据必须按照JSONL格式进行格式化，这意味着每个嵌入字典都单独写成一个JSON字符串。 

此外，为了展示过滤功能，将`restricts`键设置为每个嵌入具有不同的`class`，即`even`或`odd`。在后续匹配步骤中使用这些键来过滤结果。有关过滤的更多信息，请参阅[过滤向量匹配](https://cloud.google.com/vertex-ai/docs/vector-search/filtering)。

In [ ]:
with open("glove100.json", "w") as f:
    embeddings_formatted = [
        json.dumps(
            {
                "id": str(index),
                "embedding": [str(value) for value in embedding],
                "restricts": [
                    {
                        "namespace": "class",
                        "allow_list": ["even" if index % 2 == 0 else "odd"],
                    }
                ],
            }
        )
        + "\n"
        for index, embedding in enumerate(train)
    ]
    f.writelines(embeddings_formatted)

将训练数据上传到GCS。

In [ ]:
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/vector_search/initial/"
! gsutil cp glove100.json {EMBEDDINGS_INITIAL_URI}

创建索引##

In [ ]:
# set no.of dimensions for your embeddings
DIMENSIONS = 100
# set the dispaly name for ann index
DISPLAY_NAME = "glove_100_1"
# set the display name for brute force index
DISPLAY_NAME_BRUTE_FORCE = DISPLAY_NAME + "_brute_force"

### 创建ANN指数（用于生产使用）

创建ANN索引配置：

要了解更多关于配置索引的信息，请查看[输入数据格式和结构](https://cloud.google.com/vertex-ai/docs/vector-search/setup/format-structure)。

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    description="Glove 100 ANN index",
    labels={"label_name": "label_value"},
)

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

通过资源名称，您可以检索到现有的向量搜索索引。

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

### 创建暴力索引（用于真实数据）

暴力索引使用一种天真的暴力方法来寻找最近邻居。这种方法既不快速也不高效。因此，不建议在生产环境中使用暴力索引。它们应该用于查找“地面真实”邻居集，以便可以使用“地面真实”集来衡量调整为生产使用的索引的召回率。为了确保“苹果对苹果”比较，暴力索引的`distanceMeasureType`和`dimensions`应该与被调整为生产使用的索引的相匹配。

创建暴力索引配置：

In [ ]:
brute_force_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
    display_name=DISPLAY_NAME_BRUTE_FORCE,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Glove 100 index (brute force)",
    labels={"label_name": "label_value"},
)

In [ ]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = brute_force_index.resource_name
INDEX_BRUTE_FORCE_RESOURCE_NAME

In [ ]:
brute_force_index = aiplatform.MatchingEngineIndex(
    index_name=INDEX_BRUTE_FORCE_RESOURCE_NAME
)

## 更新索引

创建一个增量数据文件。

In [ ]:
with open("glove100_incremental.json", "w") as f:
    index = 0
    f.write(
        json.dumps(
            {
                "id": str(index),
                "embedding": [str(0) for _ in train[index]],
                "restricts": [
                    {
                        "namespace": "class",
                        "allow_list": ["even" if index % 2 == 0 else "odd"],
                    }
                ],
            }
        )
        + "\n"
    )

把增量数据文件复制到一个新的子目录。

In [ ]:
EMBEDDINGS_UPDATE_URI = f"{BUCKET_URI}/vector-search/incremental/"

In [ ]:
! gsutil cp glove100_incremental.json {EMBEDDINGS_UPDATE_URI}

创建更新索引请求。

In [ ]:
tree_ah_index = tree_ah_index.update_embeddings(
    contents_delta_uri=EMBEDDINGS_UPDATE_URI,
)

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

在您的VPC网络中创建一个索引终端点。

In [ ]:
# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
# Get the full network resource name
VPC_NETWORK_FULL = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, VPC_NETWORK)
VPC_NETWORK_FULL

In [ ]:
# Create your IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index_endpoint_for_demo",
    description="index endpoint description",
    network=VPC_NETWORK_FULL,
)

In [ ]:
INDEX_ENDPOINT_NAME = my_index_endpoint.resource_name
INDEX_ENDPOINT_NAME

部署索引

### 部署ANN索引

In [ ]:
# Set an id for your ann index deployment
DEPLOYED_INDEX_ID = "tree_ah_glove_deployed_unique"

In [ ]:
# Deploy your ann index
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

### 部署暴力指数

In [ ]:
# Set an id for your brute force index deployment
DEPLOYED_BRUTE_FORCE_INDEX_ID = "glove_brute_force_deployed_unique"

In [ ]:
# Deploy your brute force index
my_index_endpoint = my_index_endpoint.deploy_index(
    index=brute_force_index, deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID
)

my_index_endpoint.deployed_indexes

创建在线查询

在构建索引之后，您可以通过在线查询的 gRPC API（匹配服务）针对部署的索引进行查询，该查询API需在相同区域（例如，在本教程中为 'us-central1'）的虚拟机实例内进行。

`filter` 参数是一种可选的筛选嵌入向量子集的方法。在这种情况下，只返回具有设定为 `even` 的 `class` 的嵌入向量。

In [ ]:
# Use match service with a test query
response = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test[:1].tolist(),
    num_neighbors=NUM_NEIGHBOURS,
    filter=[Namespace("class", ["even"])],
)

response

### 计算召回率

使用部署的蛮力索引作为基准来计算ANN索引的召回率。您可以在单个匹配调用中运行多个查询。

In [ ]:
# Retrieve nearest neighbors for both the tree-AH index and the brute force index
tree_ah_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test[:].tolist(),
    num_neighbors=NUM_NEIGHBOURS,
)
brute_force_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID,
    queries=test[:].tolist(),
    num_neighbors=NUM_NEIGHBOURS,
)

In [ ]:
# Calculate recall by determining how many neighbors were correctly retrieved as compared to the brute force option.
recalled_neighbors = 0
for tree_ah_neighbors, brute_force_neighbors in zip(
    tree_ah_response_test, brute_force_response_test
):
    tree_ah_neighbor_ids = [neighbor.id for neighbor in tree_ah_neighbors]
    brute_force_neighbor_ids = [neighbor.id for neighbor in brute_force_neighbors]

    recalled_neighbors += len(
        set(tree_ah_neighbor_ids).intersection(brute_force_neighbor_ids)
    )

recall = recalled_neighbors / len(
    [neighbor for neighbors in brute_force_response_test for neighbor in neighbors]
)

print("Recall: {}".format(recall))

清理

要清理在此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。
您还可以通过运行以下代码手动删除您创建的资源。

In [ ]:
delete_bucket = False

# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

# Delete indexes
tree_ah_index.delete()
brute_force_index.delete()

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}